# Machine Learning Report

In [72]:
import numpy as np
from numpy import array
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

import warnings

In [73]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 85190115463408744
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6590298194
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16370867858173697102
physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


## Contents
- Loading the data
- Experimenting with the data
    - PCA
    - Regularisation
        - Using Preprocessing
        - Using MinMax Scaling
- Deep Learning Approach
- Classical Learning Approaches
    - Supervised Learning
        - K-Nearest Neighbours
        - Support Vector Machine
    - Unsupervised Learning
        - K-Means

## Loading the data

In [4]:
atlas_labels = pd.read_csv("ATLAS-labels.csv", header=None)
atlas_data = pd.read_csv("ATLAS-data.csv", header=None)

In [5]:
atlas_data.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,243.128,96.455,158.618,90.253,0.693,56.238,1.581,3.615,13.597,264.586,...,2.121,376.410,3.0,46.248,-0.958,-0.444,42.759,-1.651,-1.445,157.638
1,139.751,74.028,111.619,44.266,2.119,104.723,-0.498,2.533,9.437,168.638,...,0.447,252.698,2.0,36.798,0.269,-1.254,31.816,-1.849,0.664,68.614
2,116.217,73.136,59.239,101.850,0.625,85.539,2.979,1.922,28.049,202.636,...,1.816,271.485,2.0,82.453,2.066,-1.210,58.216,1.442,-2.288,140.669
3,64.544,0.099,28.945,341.418,2.390,358.936,-1.181,0.714,35.293,508.349,...,-2.409,470.806,3.0,331.152,-1.693,0.615,41.742,0.697,1.083,409.977
4,32.281,67.393,26.006,29.088,1.330,108.191,-0.362,0.846,48.389,152.087,...,-1.421,244.220,2.0,48.198,0.949,1.517,40.382,-0.382,-2.029,88.580


### Experimenting with the Data

#### Looking at the class balance

In [6]:
atlas_labels_counting = atlas_labels.to_numpy()
class_1_counter = 0
class_2_counter = 0
class_other = 0
for i in range(0, len(atlas_labels_counting)-1):
    if atlas_labels_counting[i] == 0:
        class_1_counter += 1
    elif atlas_labels_counting[i] == 1:
        class_2_counter += 1
    else:
        class_other += 1
print("Class 0: ", class_1_counter)
print("Class 1: ", class_2_counter)
print("Class Other: ", class_other)

Class 0:  5311
Class 1:  4688
Class Other:  0


#### Picking a Preprocessor

In [7]:
# 0 for Standard Scaler
# 1 for PCA
# 2 for "preprocessing"
# 3 for MinMax Scaler
preprocessor_setting = 0

#### Standard Scaler

In [8]:
if preprocessor_setting == 0:
    scaler = StandardScaler()
    scaler.fit(atlas_data)
    atlas_processed = scaler.transform(atlas_data)

#### Principal Component Analysis

In [9]:
if preprocessor_setting == 1:
    pca = PCA(.95)
    pca.fit(atlas_processed)
    atlas_processed = pca.transform(atlas_processed)
    atlas_processed.shape

#### Reguralization

##### Using preprocessing

In [10]:
if preprocessor_setting == 2:
    atlas_processed = preprocessing.scale(atlas_data)

##### Using MinMax Scaling

In [11]:
if preprocessor_setting == 3:
    scaling = MinMaxScaler(feature_range=(-1,1)).fit(atlas_data)
    atlas_processed = scaling.transform(atlas_data)

## Deep Learning Approach

### Neural Network

This section involves using a deep neural network (more than 1 hidden layer) to perform the classification task.

In [82]:
NN_Data = atlas_processed
NN_Labels = atlas_labels

In [83]:
nn_train_values, nn_test_values, nn_train_labels, nn_test_labels = train_test_split(NN_Data, NN_Labels, test_size=0.2)

In [84]:
nn_train_labs = nn_train_labels
nn_test_labs = nn_test_labels

nn_train_labels = keras.utils.to_categorical(nn_train_labels)
nn_test_labels = keras.utils.to_categorical(nn_test_labels)

Declaring some values that can be changed to experiment with the network whilst creating.

In [16]:
epochs = 1000

In [17]:
inputs=keras.Input(shape=(30,))
hidden_layer_1=layers.Dense(512,activation='relu')
hidden_layer_2=layers.Dense(256,activation='sigmoid')
hidden_layer_3=layers.Dense(128,activation='sigmoid')
hidden_layer_4=layers.Dense(128,activation='relu')
output_layer=layers.Dense(2,activation='sigmoid')

z=hidden_layer_1(inputs)
z=hidden_layer_2(z)
z=hidden_layer_3(z)
z=hidden_layer_4(z)
output=output_layer(z)

model=keras.Model(inputs=inputs,outputs=output)

In [18]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')
model.fit(nn_train_values,nn_train_labels,epochs=epochs,verbose=1,validation_split=0.2)

Train on 6400 samples, validate on 1600 samples
Epoch 1/1000
6400/6400 [==============================] - 2s 319us/sample - loss: 0.1731 - accuracy: 0.7370 - val_loss: 0.1187 - val_accuracy: 0.8294
Epoch 2/1000
6400/6400 [==============================] - 0s 53us/sample - loss: 0.1305 - accuracy: 0.8128 - val_loss: 0.1140 - val_accuracy: 0.8363
Epoch 3/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.1212 - accuracy: 0.8278 - val_loss: 0.1127 - val_accuracy: 0.8363
Epoch 4/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.1182 - accuracy: 0.8364 - val_loss: 0.1101 - val_accuracy: 0.8494
Epoch 5/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.1144 - accuracy: 0.8433 - val_loss: 0.1118 - val_accuracy: 0.8381
Epoch 6/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.1124 - accuracy: 0.8452 - val_loss: 0.1174 - val_accuracy: 0.8419
Epoch 7/1000
6400/6400 [==============================] -

6400/6400 [==============================] - 0s 48us/sample - loss: 0.0315 - accuracy: 0.9648 - val_loss: 0.1531 - val_accuracy: 0.8238
Epoch 56/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0304 - accuracy: 0.9653 - val_loss: 0.1534 - val_accuracy: 0.8225
Epoch 57/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0293 - accuracy: 0.9664 - val_loss: 0.1522 - val_accuracy: 0.8269
Epoch 58/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0309 - accuracy: 0.9631 - val_loss: 0.1554 - val_accuracy: 0.8250
Epoch 59/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0294 - accuracy: 0.9667 - val_loss: 0.1559 - val_accuracy: 0.8200
Epoch 60/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0283 - accuracy: 0.9680 - val_loss: 0.1469 - val_accuracy: 0.8338
Epoch 61/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0279 - accuracy: 0.9681 - val_

6400/6400 [==============================] - 0s 53us/sample - loss: 0.0129 - accuracy: 0.9858 - val_loss: 0.1646 - val_accuracy: 0.8181
Epoch 110/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 0.0141 - accuracy: 0.9845 - val_loss: 0.1636 - val_accuracy: 0.8163
Epoch 111/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 0.0138 - accuracy: 0.9842 - val_loss: 0.1625 - val_accuracy: 0.8188
Epoch 112/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 0.0155 - accuracy: 0.9823 - val_loss: 0.1585 - val_accuracy: 0.8238
Epoch 113/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 0.0151 - accuracy: 0.9827 - val_loss: 0.1619 - val_accuracy: 0.8188
Epoch 114/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 0.0120 - accuracy: 0.9872 - val_loss: 0.1608 - val_accuracy: 0.8181
Epoch 115/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0114 - accuracy: 0.9877 

6400/6400 [==============================] - 0s 49us/sample - loss: 0.0072 - accuracy: 0.9927 - val_loss: 0.1579 - val_accuracy: 0.8263
Epoch 164/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0072 - accuracy: 0.9928 - val_loss: 0.1580 - val_accuracy: 0.8294
Epoch 165/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0071 - accuracy: 0.9928 - val_loss: 0.1583 - val_accuracy: 0.8281
Epoch 166/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0069 - accuracy: 0.9931 - val_loss: 0.1601 - val_accuracy: 0.8238
Epoch 167/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0066 - accuracy: 0.9934 - val_loss: 0.1591 - val_accuracy: 0.8275
Epoch 168/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0066 - accuracy: 0.9934 - val_loss: 0.1590 - val_accuracy: 0.8294
Epoch 169/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0066 - accuracy: 0.9934 

6400/6400 [==============================] - 0s 50us/sample - loss: 0.0090 - accuracy: 0.9908 - val_loss: 0.1678 - val_accuracy: 0.8106
Epoch 218/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 0.0073 - accuracy: 0.9925 - val_loss: 0.1652 - val_accuracy: 0.8213
Epoch 219/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0065 - accuracy: 0.9934 - val_loss: 0.1707 - val_accuracy: 0.8094
Epoch 220/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0066 - accuracy: 0.9934 - val_loss: 0.1630 - val_accuracy: 0.8244
Epoch 221/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0064 - accuracy: 0.9936 - val_loss: 0.1631 - val_accuracy: 0.8244
Epoch 222/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0064 - accuracy: 0.9936 - val_loss: 0.1633 - val_accuracy: 0.8219
Epoch 223/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 0.0064 - accuracy: 0.9936 

...

6400/6400 [==============================] - 0s 47us/sample - loss: 0.0030 - accuracy: 0.9970 - val_loss: 0.1696 - val_accuracy: 0.8225
Epoch 758/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0030 - accuracy: 0.9970 - val_loss: 0.1742 - val_accuracy: 0.8169
Epoch 759/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0133 - accuracy: 0.9852 - val_loss: 0.1792 - val_accuracy: 0.8044
Epoch 760/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0133 - accuracy: 0.9839 - val_loss: 0.1849 - val_accuracy: 0.7987
Epoch 761/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0074 - accuracy: 0.9909 - val_loss: 0.1644 - val_accuracy: 0.8200
Epoch 762/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 0.0045 - accuracy: 0.9950 - val_loss: 0.1713 - val_accuracy: 0.8138
Epoch 763/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0031 - accuracy: 0.9969 

6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1748 - val_accuracy: 0.8169
Epoch 812/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1749 - val_accuracy: 0.8169
Epoch 813/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1749 - val_accuracy: 0.8169
Epoch 814/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1750 - val_accuracy: 0.8169
Epoch 815/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1750 - val_accuracy: 0.8169
Epoch 816/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1750 - val_accuracy: 0.8169
Epoch 817/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0027 - accuracy: 0.9973 

6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1757 - val_accuracy: 0.8194
Epoch 866/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1757 - val_accuracy: 0.8194
Epoch 867/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1757 - val_accuracy: 0.8194
Epoch 868/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1757 - val_accuracy: 0.8194
Epoch 869/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1757 - val_accuracy: 0.8194
Epoch 870/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1757 - val_accuracy: 0.8194
Epoch 871/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0027 - accuracy: 0.9973 

6400/6400 [==============================] - 0s 47us/sample - loss: 0.0028 - accuracy: 0.9972 - val_loss: 0.1744 - val_accuracy: 0.8169
Epoch 920/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0028 - accuracy: 0.9972 - val_loss: 0.1744 - val_accuracy: 0.8175
Epoch 921/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0028 - accuracy: 0.9972 - val_loss: 0.1745 - val_accuracy: 0.8175
Epoch 922/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0028 - accuracy: 0.9972 - val_loss: 0.1745 - val_accuracy: 0.8175
Epoch 923/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0028 - accuracy: 0.9972 - val_loss: 0.1745 - val_accuracy: 0.8175
Epoch 924/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0028 - accuracy: 0.9972 - val_loss: 0.1746 - val_accuracy: 0.8175
Epoch 925/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0028 - accuracy: 0.9972 

6400/6400 [==============================] - 0s 48us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1645 - val_accuracy: 0.8231
Epoch 974/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1647 - val_accuracy: 0.8231
Epoch 975/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1649 - val_accuracy: 0.8231
Epoch 976/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1650 - val_accuracy: 0.8231
Epoch 977/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1651 - val_accuracy: 0.8231
Epoch 978/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0027 - accuracy: 0.9973 - val_loss: 0.1653 - val_accuracy: 0.8231
Epoch 979/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0027 - accuracy: 0.9973 

In [19]:
_, train_acc = model.evaluate(nn_train_values, nn_train_labels, verbose=0)
_, test_acc = model.evaluate(nn_test_values, nn_test_labels, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.962, Test: 0.800


### Ensemble Neural Network

Function to fit a model

In [38]:
def fit_model(train_data, train_labels):
    inputs=keras.Input(shape=(30,))
    hidden_layer_1=layers.Dense(512,activation='relu')
    hidden_layer_2=layers.Dense(256,activation='sigmoid')
    hidden_layer_3=layers.Dense(128,activation='sigmoid')
    hidden_layer_4=layers.Dense(128,activation='relu')
    output_layer=layers.Dense(2,activation='sigmoid')

    z=hidden_layer_1(inputs)
    z=hidden_layer_2(z)
    z=hidden_layer_3(z)
    z=hidden_layer_4(z)
    output=output_layer(z)
    
    model=keras.Model(inputs=inputs,outputs=output)
    
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')
    model.fit(train_data,train_labels,epochs=epochs,verbose=1,validation_split=0.2)
    return model

Declare and fit a number of models

In [39]:
num_models = 10
models = [fit_model(nn_train_values, nn_train_labels) for _ in range(num_models)]

Train on 6400 samples, validate on 1600 samples
Epoch 1/1000
6400/6400 [==============================] - 1s 136us/sample - loss: 0.5010 - accuracy: 0.7550 - val_loss: 0.3975 - val_accuracy: 0.8125
Epoch 2/1000
6400/6400 [==============================] - 0s 54us/sample - loss: 0.3963 - accuracy: 0.8208 - val_loss: 0.3743 - val_accuracy: 0.8294
Epoch 3/1000
6400/6400 [==============================] - 0s 53us/sample - loss: 0.3752 - accuracy: 0.8316 - val_loss: 0.3695 - val_accuracy: 0.8281
Epoch 4/1000
6400/6400 [==============================] - 0s 53us/sample - loss: 0.3591 - accuracy: 0.8430 - val_loss: 0.3680 - val_accuracy: 0.8288
Epoch 5/1000
6400/6400 [==============================] - 0s 54us/sample - loss: 0.3519 - accuracy: 0.8434 - val_loss: 0.3675 - val_accuracy: 0.8375
Epoch 6/1000
6400/6400 [==============================] - 0s 54us/sample - loss: 0.3458 - accuracy: 0.8483 - val_loss: 0.3659 - val_accuracy: 0.8319
Epoch 7/1000
6400/6400 [==============================] -

6400/6400 [==============================] - 0s 53us/sample - loss: 0.0434 - accuracy: 0.9850 - val_loss: 1.1427 - val_accuracy: 0.8037
Epoch 56/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 0.0472 - accuracy: 0.9812 - val_loss: 1.1626 - val_accuracy: 0.8106
Epoch 57/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 0.0448 - accuracy: 0.9847 - val_loss: 1.1386 - val_accuracy: 0.7969
Epoch 58/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 0.0339 - accuracy: 0.9873 - val_loss: 1.2320 - val_accuracy: 0.7894
Epoch 59/1000
6400/6400 [==============================] - 0s 55us/sample - loss: 0.0268 - accuracy: 0.9912 - val_loss: 1.3031 - val_accuracy: 0.8138
Epoch 60/1000
6400/6400 [==============================] - 0s 53us/sample - loss: 0.0372 - accuracy: 0.9866 - val_loss: 1.1860 - val_accuracy: 0.8012
Epoch 61/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 0.0305 - accuracy: 0.9887 - val_

Epoch 109/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 6.1650e-06 - accuracy: 1.0000 - val_loss: 1.9827 - val_accuracy: 0.8069
Epoch 110/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 5.4596e-06 - accuracy: 1.0000 - val_loss: 1.9977 - val_accuracy: 0.8069
Epoch 111/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 4.8443e-06 - accuracy: 1.0000 - val_loss: 2.0022 - val_accuracy: 0.8069
Epoch 112/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 4.3056e-06 - accuracy: 1.0000 - val_loss: 2.0162 - val_accuracy: 0.8069
Epoch 113/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 3.8193e-06 - accuracy: 1.0000 - val_loss: 2.0346 - val_accuracy: 0.8056
Epoch 114/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 3.4227e-06 - accuracy: 1.0000 - val_loss: 2.0383 - val_accuracy: 0.8069
Epoch 115/1000
6400/6400 [==============================] - 0s 51us/sa

Epoch 162/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 0.0281 - accuracy: 0.9912 - val_loss: 1.1698 - val_accuracy: 0.7962
Epoch 163/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 0.0203 - accuracy: 0.9933 - val_loss: 1.2963 - val_accuracy: 0.8006
Epoch 164/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 0.0318 - accuracy: 0.9905 - val_loss: 1.2084 - val_accuracy: 0.7981
Epoch 165/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 0.0168 - accuracy: 0.9939 - val_loss: 1.4009 - val_accuracy: 0.7994
Epoch 166/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 0.0159 - accuracy: 0.9953 - val_loss: 1.3768 - val_accuracy: 0.7950
Epoch 167/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 0.0282 - accuracy: 0.9897 - val_loss: 1.3031 - val_accuracy: 0.8012
Epoch 168/1000
6400/6400 [==============================] - 0s 53us/sample - loss: 0.0230 - ac

6400/6400 [==============================] - 0s 52us/sample - loss: 3.9661e-06 - accuracy: 1.0000 - val_loss: 2.0333 - val_accuracy: 0.8044
Epoch 216/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 3.5295e-06 - accuracy: 1.0000 - val_loss: 2.0416 - val_accuracy: 0.8044
Epoch 217/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 3.1379e-06 - accuracy: 1.0000 - val_loss: 2.0562 - val_accuracy: 0.8037
Epoch 218/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 2.7997e-06 - accuracy: 1.0000 - val_loss: 2.0667 - val_accuracy: 0.8037
Epoch 219/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 2.4881e-06 - accuracy: 1.0000 - val_loss: 2.0790 - val_accuracy: 0.8037
Epoch 220/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 2.2210e-06 - accuracy: 1.0000 - val_loss: 2.0889 - val_accuracy: 0.8037
Epoch 221/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.

...

Epoch 769/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 770/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 771/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 772/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 773/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 774/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 775/1000
6400/6400 [==============================] - 0s 52us/sa

Epoch 822/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 823/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 824/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 825/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 826/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 827/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 828/1000
6400/6400 [==============================] - 0s 51us/sa

Epoch 875/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 876/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 877/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 878/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 879/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 880/1000
6400/6400 [==============================] - 0s 55us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 881/1000
6400/6400 [==============================] - 0s 52us/sa

Epoch 928/1000
6400/6400 [==============================] - 0s 57us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 929/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 930/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 931/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 932/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 933/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 934/1000
6400/6400 [==============================] - 0s 51us/sa

Epoch 981/1000
6400/6400 [==============================] - 0s 53us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 982/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 983/1000
6400/6400 [==============================] - 0s 51us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 984/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 985/1000
6400/6400 [==============================] - 0s 52us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 986/1000
6400/6400 [==============================] - 0s 53us/sample - loss: 1.1921e-07 - accuracy: 1.0000 - val_loss: 2.6782 - val_accuracy: 0.7975
Epoch 987/1000
6400/6400 [==============================] - 0s 52us/sa

Function to predict using the ensemble models

In [77]:
def ensemble_predictions(models, testX):
    # make predictions
    yhats = [model.predict(testX) for model in models]
    yhats = array(yhats)
    # sum across ensemble members
    summed = np.sum(yhats, axis=0)
    # argmax across classes
    result = np.argmax(summed, axis=1)
    return result

In [80]:
def evaluate_n_members(models, num_models, testX, testy):
    # select a subset of members
    subset = models[:num_models]
    # make prediction
    yhat = ensemble_predictions(subset, testX)
    # calculate accuracy
    return accuracy_score(testy, yhat)

In [93]:
# evaluate different numbers of ensembles on hold out set
single_scores, ensemble_scores = list(), list()
for i in range(1, len(models)+1):
    # evaluate model with i members
    ensemble_score = evaluate_n_members(models, i, nn_test_values, nn_test_labs)
    _, single_score = models[i-1].evaluate(nn_test_values, nn_test_labels, verbose=0)
    # summarize this step
    print('> %d: single=%.3f, ensemble=%.3f' % (i, single_score, ensemble_score))
    ensemble_scores.append(ensemble_score)
    single_scores.append(single_score)
# summarize average accuracy of a single final model
print('Accuracy %.3f (%.3f)' % (np.mean(single_scores), np.std(single_scores)))

[1 0 0 ... 1 1 0]
      0
8975  1
132   0
5921  0
3814  1
1710  0
...  ..
4721  0
9131  0
4899  1
6526  1
5722  0

[2000 rows x 1 columns]
> 1: single=0.924, ensemble=0.924
[1 0 0 ... 1 1 0]
      0
8975  1
132   0
5921  0
3814  1
1710  0
...  ..
4721  0
9131  0
4899  1
6526  1
5722  0

[2000 rows x 1 columns]
> 2: single=0.929, ensemble=0.928
[1 0 0 ... 1 1 0]
      0
8975  1
132   0
5921  0
3814  1
1710  0
...  ..
4721  0
9131  0
4899  1
6526  1
5722  0

[2000 rows x 1 columns]
> 3: single=0.914, ensemble=0.925
[1 0 0 ... 1 1 0]
      0
8975  1
132   0
5921  0
3814  1
1710  0
...  ..
4721  0
9131  0
4899  1
6526  1
5722  0

[2000 rows x 1 columns]
> 4: single=0.923, ensemble=0.925
[1 0 0 ... 1 1 0]
      0
8975  1
132   0
5921  0
3814  1
1710  0
...  ..
4721  0
9131  0
4899  1
6526  1
5722  0

[2000 rows x 1 columns]
> 5: single=0.923, ensemble=0.926
[1 0 0 ... 1 1 0]
      0
8975  1
132   0
5921  0
3814  1
1710  0
...  ..
4721  0
9131  0
4899  1
6526  1
5722  0

[2000 rows x 1 colum

## Classical Approaches

### Supervised

#### K-Nearest Neigbours

In [5]:
KNN_Data = atlas_processed
KNN_Labels = atlas_labels.to_numpy()

In [6]:
knn_train_values, knn_test_values, knn_train_labels, knn_test_labels = train_test_split(KNN_Data, KNN_Labels, test_size=0.2)

In [7]:
search_max_val = 10

In [8]:
warnings.simplefilter('ignore')
knn_model = KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree')
grid = {"n_neighbors":range(1,search_max_val)}
knn_model_cv = GridSearchCV(knn_model, grid, cv=10, iid=False, n_jobs=-1)
knn_model_cv.fit(KNN_Data, KNN_Labels)
knn_model_cv.best_params_

{'n_neighbors': 8}

In [9]:
knn_model_final = KNeighborsClassifier(n_neighbors=knn_model_cv.best_params_['n_neighbors'], algorithm='ball_tree')
knn_model_final.fit(knn_train_values, knn_train_labels)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='uniform')

In [10]:
knn_score = knn_model_final.score(knn_test_values, knn_test_labels)

In [11]:
print("Score for KNN is ",knn_score)

Score for KNN is  0.6815


#### Support Vector Machine - SVM

In [12]:
SVM_Data = atlas_processed
SVM_Labels = atlas_labels.to_numpy()

In [13]:
svm_train_values, svm_test_values, svm_train_labels, svm_test_labels = train_test_split(SVM_Data, SVM_Labels, test_size=0.2)

In [14]:
search_max_val = 10

In [15]:
svm_model_lin = SVC(kernel="linear")


In [16]:
grid = {"C":range(1,search_max_val)}

svm_model_cv = GridSearchCV(svm_model_lin, grid, cv=10, iid=False, n_jobs=-1)
svm_model_cv.fit(svm_train_values, svm_train_labels)
svm_model_cv.best_params_

{'C': 5}

In [17]:
svm_model_lin_final = SVC(C=svm_model_cv.best_params_['C'],kernel="linear")
svm_model_lin_final.fit(svm_train_values, svm_train_labels)
svm_lin_score = svm_model_lin_final.score(svm_test_values, svm_test_labels)
print("Score for Linear SVM is ",svm_lin_score)

Score for Linear SVM is  0.733


In [18]:
svm_model_rbf = SVC(kernel="rbf")

In [19]:
grid = {"C":range(1,search_max_val)}

svm_model_cv = GridSearchCV(svm_model_rbf, grid, cv=10, iid=False)
svm_model_cv.fit(svm_train_values, svm_train_labels)
svm_model_cv.best_params_

{'C': 9}

In [21]:
svm_model_rbf_final = SVC(C=svm_model_cv.best_params_['C'],kernel="linear")
svm_model_rbf_final.fit(svm_train_values, svm_train_labels)
svm_rbf_score = svm_model_rbf_final.score(svm_test_values, svm_test_labels)
print("Score for RBF SVM is ",svm_rbf_score)

Score for RBF SVM is  0.733


### Unsupervised

#### K-Means Clustering

In [22]:
KMEANS_Data = atlas_processed
KMEAN_Labels = atlas_labels.to_numpy()

In [23]:
numClusters = 2
iterations = 100

In [26]:
kmeans_model = KMeans(numClusters, max_iter=iterations)

In [27]:
kmeans_model.fit(KMEANS_Data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [44]:
cpy = atlas_labels.to_numpy()
counter = 0
for i in range(0, len(kmeans_model.labels_)):
    if(kmeans_model.labels_[i] == cpy[i]):
        counter += 1
score = counter / len(kmeans_model.labels_)
print(score)

0.5771
